In [261]:
import os
import pprint
from collections import defaultdict, namedtuple, UserList
from copy import deepcopy
import abc
from typing import Set, Any

import lxml
import lxml.html
import lxml.etree
from graphviz import Digraph
from similarity.normalized_levenshtein import NormalizedLevenshtein


normalized_levenshtein = NormalizedLevenshtein()
NODE_NAME_ATTRIB = '___tag_name___'
HIERARCHICAL = 'hierarchical'
SEQUENTIAL = 'sequential'


class WithBasicFormat(object):
    
    def _extra_format(self, format_spec):
        raise NotImplementedError()
    
    def __format__(self, format_spec):
        if format_spec == "!s":
            return str(self)
        elif format_spec in ("!r", ""):
            return repr(self)
        else:
            try:
                return self._extra_format(format_spec)
            except NotImplementedError:
                raise TypeError("unsupported format string passed to {}.__format__".format(type(self).__name__))


class GNode(namedtuple("GNode", ["parent", "start", "end"]), WithBasicFormat):
    """Generalized Node - start/end are indexes of sibling nodes relative to their parent node."""
    
    def __str__(self):
        return "GN({start:>2}, {end:>2})".format(start=self.start, end=self.end)
    
    def __len__(self):
        return self.size
    
    def _extra_format(self, format_spec):
        if format_spec == '!S':
            return "GN({parent}, {start:>2}, {end:>2})".format(parent=self.parent, start=self.start, end=self.end)
        else:
            raise NotImplementedError()
    
    @property
    def size(self):
        return self.end - self.start

    
class GNodePair(namedtuple("GNodePair", ["left",  "right"]), WithBasicFormat):
    """Generalized Node Pair - pair of adjacent GNodes, used for stocking the edit distances between them."""

    def __str__(self):
        return "{left:!s} - {right:!s}".format(left=self.left, right=self.right)


# noinspection PyArgumentList
class DataRegion(
    namedtuple("DataRegion", ["parent", "gnode_size", "first_gnode_start_index", "n_nodes_covered"]), WithBasicFormat
):
    """Data Region - a continuous sequence of GNode's."""
    
    def __str__(self):
        return "DR({0}, {1}, {2})".format(self.gnode_size, self.first_gnode_start_index, self.n_nodes_covered)
    
    def __contains__(self, child_index):
        msg = "DataRegion contains the indexes of a node relative to its parent list of children. " \
              "Type `{}` not supported.".format(type(child_index).__name__)
        assert isinstance(child_index, int), msg
        return self.first_gnode_start_index <= child_index <= self.last_covered_tag_index
    
    def __iter__(self):
        self._iter_i = 0
        return self

    def __next__(self):
        if self._iter_i < self.n_gnodes:
            start = self.first_gnode_start_index + self._iter_i * self.gnode_size
            end = start + self.gnode_size
            gnode = GNode(self.parent, start, end)
            self._iter_i += 1
            return gnode
        else:
            raise StopIteration
    
    def _extra_format(self, format_spec):
        if format_spec == '!S':
            return "DR({0}, {1}, {2}, {3})".format(
                self.parent, self.gnode_size, self.first_gnode_start_index, self.n_nodes_covered
            )
        else:
            raise NotImplementedError()
        
    @classmethod
    def empty(cls):
        return cls(None, None, None, 0)
    
    @classmethod
    def binary_from_last_gnode(cls, gnode):
        gnode_size = gnode.end - gnode.start
        return cls(gnode.parent, gnode_size, gnode.start - gnode_size, 2 * gnode_size)

    @property
    def is_empty(self):
        return self[0] is None
    
    @property
    def n_gnodes(self):
        return self.n_nodes_covered // self.gnode_size
    
    @property
    def last_covered_tag_index(self):
        return self.first_gnode_start_index + self.n_nodes_covered - 1
    
    def extend_one_gnode(self):
        return self.__class__(
            self.parent, self.gnode_size, self.first_gnode_start_index, self.n_nodes_covered + self.gnode_size
        )


class DataRecord(UserList, WithBasicFormat):
    
    def __hash__(self):
        return hash(tuple(self))

    def __repr__(self):
        return "DataRecord({})".format(", ".join([repr(gn) for gn in self.data]))
    
    def __str__(self):
        return "DataRecord({})".format(", ".join([str(gn) for gn in self.data]))


def open_html_document(directory, file):
    directory = os.path.abspath(directory)
    filepath = os.path.join(directory, file)
    with open(filepath, 'r') as file:
        html_document = lxml.html.fromstring(
            lxml.etree.tostring(lxml.html.parse(file), method='html')
        )
    return html_document


def html_to_dot_sequential_name(html, with_text=False):
    graph = Digraph(name='html')
    tag_counts = defaultdict(int)
    
    def add_node(html_node):
        tag = html_node.tag
        tag_sequential = tag_counts[tag]
        tag_counts[tag] += 1
        node_name = "{}-{}".format(tag, tag_sequential)
        graph.node(node_name, node_name)
        
        if len(html_node) > 0:
            for child in html_node.iterchildren():
                child_name = add_node(child)
                graph.edge(node_name, child_name)
        elif with_text:
            child_name = "-".join([node_name, "txt"])
            graph.node(child_name, html_node.text)
            graph.edge(node_name, child_name)
        return node_name
    add_node(html)
    return graph


def html_to_dot_hierarchical_name(html, with_text=False):
    graph = Digraph(name='html')
    
    def add_node(html_node, parent_suffix, brotherhood_index):
        tag = html_node.tag
        if parent_suffix is None and brotherhood_index is None:
            node_suffix = ""
            node_name = tag
        else:
            node_suffix = (
                "-".join([parent_suffix, str(brotherhood_index)]) 
                if parent_suffix else 
                str(brotherhood_index)
            )
            node_name = "{}-{}".format(tag, node_suffix)
        graph.node(node_name, node_name, path=node_suffix)
        
        if len(html_node) > 0:
            for child_index, child in enumerate(html_node.iterchildren()):
                child_name = add_node(child, node_suffix, child_index)
                graph.edge(node_name, child_name)
        elif with_text:
            child_name = "-".join([node_name, "txt"])
            child_path = "-".join([node_suffix, "txt"])
            graph.node(child_name, html_node.text, path=child_path)
            graph.edge(node_name, child_name)
        return node_name
    add_node(html, None, None)
    return graph


def html_to_dot(html, name_option='hierarchical', with_text=False):
    if name_option == SEQUENTIAL:
        return html_to_dot_sequential_name(html, with_text=with_text)
    elif name_option == HIERARCHICAL:
        return html_to_dot_hierarchical_name(html, with_text=with_text)
    else:
        raise Exception('No name option `{}`'.format(name_option))


class FormatPrinter(pprint.PrettyPrinter):

    def __init__(self, formats):
        super(FormatPrinter, self).__init__()
        self.formats = formats

    def format(self, obj, ctx, max_lvl, lvl):
        obj_type = type(obj)
        if obj_type in self.formats:
            type_format = self.formats[obj_type]
            return "{{0:{}}}".format(type_format).format(obj), 1, 0
        return pprint.PrettyPrinter.format(self, obj, ctx, max_lvl, lvl)


In [262]:
class MDRVerbosity(
    namedtuple("MDRVerbosity", "compute_distances find_data_regions identify_data_records")
):
    @classmethod
    def absolute_silent(cls):
        return cls(None, None, None)
    
    @property
    def is_absolute_silent(self):
        return any(val is None for val in self)
    
    @classmethod
    def silent(cls):
        return cls(False, False, False)
    
    @classmethod
    def only_compute_distances(cls):
        return cls(True, False, False)

    @classmethod
    def only_find_data_regions(cls):
        return cls(False, True, False)

    @classmethod
    def only_identify_data_records(cls):
        return cls(False, False, True)
    

class UsedMDRException(Exception):
     default_message = "This MDR instance has already been used. Please instantiate another one."

     def __init__(self, *args, **kwargs):
         if not (args or kwargs): 
             args = (self.default_message,)
         super(Exception, self).__init__(*args, **kwargs)


# noinspection PyArgumentList
class MDR:
    """
    Notation: 
        gn = gnode = generalized node
        dr = data region
    """

    MINIMUM_DEPTH = 3

    def __init__(self, max_tag_per_gnode, edit_distance_threshold, verbose=None, keep_all_data_regions=False):
        self.max_tag_per_gnode = max_tag_per_gnode
        self.edit_distance_threshold = edit_distance_threshold
        self.keep_all_data_regions = keep_all_data_regions
        self._verbose = verbose if verbose is not None else MDRVerbosity.silent()
        self._phase = None
        self._used = False

    def _debug(self, msg, tabs=0, force=False):
        if self._verbose.is_absolute_silent:
            return
        
        if self._verbose[self._phase] or force:
            if type(msg) == str:
                print(tabs * '\t' + msg)
            else:
                FormatPrinter({float: ".2f", GNode: "!s", GNodePair: "!s", DataRegion: "!s"}).pprint(msg)
                
    def _debug_phase(self, phase):
        if self._phase is not None:
            title = " END PHASE {} ({}) ".format(MDRVerbosity._fields[self._phase].upper(), self._phase)
            self._debug(">" * 20 + title + "<" * 20 + "\n\n", force=True)
        
        self._phase = phase
        if self._phase <= 2:
            title = " START PHASE {} ({}) ".format(MDRVerbosity._fields[self._phase].upper(), self._phase)
            self._debug(">" * 20 + title + "<" * 20, force=True)

    @staticmethod
    def depth(node):
        d = 0
        while node is not None:
            d += 1
            node = node.getparent()
        return d

    @staticmethod
    def gnode_to_string(list_of_nodes):
        return " ".join([
            lxml.etree.tostring(child).decode('utf-8') for child in list_of_nodes
        ])
    
    def __call__(self, root):
        if self._used:
            raise UsedMDRException()
        self._used = True
        
        # todo(improvement) change the other naming method to use this 
        class NodeNamer(object):
            
            def __init__(self):
                self.tag_counts = defaultdict(int)
                self.map = {}
            
            def __call__(self, node, *args, **kwargs):
                if NODE_NAME_ATTRIB in node.attrib:
                    return node.attrib[NODE_NAME_ATTRIB]
                # each tag is named sequentially
                tag = node.tag
                tag_sequential = self.tag_counts[tag]
                self.tag_counts[tag] += 1
                node_name = "{0}-{1:0>5}".format(tag, tag_sequential)
                node.set(NODE_NAME_ATTRIB, node_name)
                return node_name
            
            @staticmethod
            def cleanup_tag_name(node):
                del node.attrib[NODE_NAME_ATTRIB]
                
        self._debug_phase(0)
        self.distances = {}
        self.node_namer = NodeNamer()
        self._compute_distances(root)
        
        # todo(prod): remove this 
        self._debug("\n\nself.distances\n", force=True)
        self._debug(self.distances, force=True)
        self._debug("\n\n", force=True)

        self._debug_phase(1)
        self.data_regions = {}  # {node_name(str): set(GNode)}  only retains the max data regions
        self._all_data_regions_found = defaultdict(set)  # retains all of them for debug purposes
        self._checked_gnode_pairs = defaultdict(set)
        
        self._find_data_regions(root)
        
        self._checked_gnode_pairs = dict(self._checked_gnode_pairs)
        self._all_data_regions_found = dict(self._all_data_regions_found)
        
        # todo(prod): remove this 
        self._debug("\n\nself.data_regions\n", force=True)
        self._debug(self.data_regions, force=True)
        self._debug("\n\n", force=True)
        
        self._debug_phase(2)
        self.data_records = set()
        def get_node(node_name):
            tag = node_name.split("-")[0]
            # todo add some security stuff here???
            node = root.xpath("//{tag}[@___tag_name___='{node_name}']".format(tag=tag, node_name=node_name))[0]
            return node
        self._find_data_records(get_node)
        
        # todo(prod): remove this 
        self._debug("\n\nself.data_records\n", force=True)
        self.data_records = sorted(self.data_records)
        self._debug(self.data_records, force=True)
        self._debug("\n\n", force=True)
        
        self._debug_phase(3)
        # tdod cleanup aattrb ???

        return self.data_records

    def _compute_distances(self, node):
        
        # !!! ATTENTION !!! this modifies the input HTML element by adding an attribute
        # todo: remember, in the last phase, to clear the `TAG_NAME_ATTRIB` from all tags
        node_name = self.node_namer(node)
        node_depth = MDR.depth(node)
        self._debug("in _compute_distances of `{}` (depth={})".format(node_name, node_depth))

        if node_depth >= MDR.MINIMUM_DEPTH:
            
            # get all possible distances of the n-grams of children
            # {gnode_size: {GNode: float}}
            distances = self._compare_combinations(node.getchildren())
            
            # todo(prod): remove this 
            self._debug("`{}` distances".format(node_name))
            self._debug(distances)
    
        else:
            self._debug("skipped (less than min depth = {})".format(self.MINIMUM_DEPTH), 1)
            distances = None
        
        self.distances[node_name] = distances

        # todo(prod): remove this 
        self._debug("\n\n")

        for child in node:
            self._compute_distances(child)

    def _compare_combinations(self, node_list):
        """
        Notation: gnode = "generalized node"
        :returns
            {gnode_size: {GNode: float}}
        """

        self._debug("in _compare_combinations")

        if not node_list:
            self._debug("empty list --> return {}")
            return {}

        # {gnode_size: {GNode: float}}
        distances = defaultdict(dict)
        n_nodes = len(node_list)
        parent = node_list[0].getparent()
        parent_name = self.node_namer(parent)
        self._debug("n_nodes: {}".format(n_nodes))

        # 1) for (i = 1; i <= K; i++)  /* start from each node */
        for starting_tag in range(1, self.max_tag_per_gnode + 1):
            self._debug('starting_tag (i): {}'.format(starting_tag), 1)

            # 2) for (j = i; j <= K; j++) /* comparing different combinations */
            for gnode_size in range(starting_tag, self.max_tag_per_gnode + 1):  # j
                self._debug('gnode_size (j): {}'.format(gnode_size), 2)

                # 3) if NodeList[i+2*j-1] exists then
                there_are_pairs_to_look = (starting_tag + 2 * gnode_size - 1) < n_nodes + 1
                if there_are_pairs_to_look:  # +1 for pythons open set notation
                    self._debug(" ")  # todo(prod): remove this
                    self._debug(">>> if 1: there_are_pairs_to_look <<<", 3)
                    
                    # 4) St = i;
                    left_gnode_start = starting_tag - 1  # st

                    # 5) for (k = i+j; k < Size(NodeList); k+j)
                    for right_gnode_start in range(starting_tag + gnode_size - 1, n_nodes, gnode_size):  # k
                        self._debug('left_gnode_start (st): {}'.format(left_gnode_start), 4)
                        self._debug('right_gnode_start (k): {}'.format(right_gnode_start), 4)

                        # 6)  if NodeList[k+j-1] exists then
                        right_gnode_exists = right_gnode_start + gnode_size < n_nodes + 1
                        if right_gnode_exists:
                            self._debug(" ")  # todo(prod): remove this
                            self._debug(">>> if 2: right_gnode_exists <<<", 5)
                            
                            # todo(improvement): avoid recomputing strings?
                            # todo(improvement): avoid recomputing edit distances?
                            # todo(improvement): check https://pypi.org/project/strsim/ ?

                            # NodeList[St..(k-1)]
                            left_gnode = GNode(parent_name, left_gnode_start, right_gnode_start)
                            left_gnode_nodes = node_list[left_gnode.start:left_gnode.end]
                            left_gnode_str = MDR.gnode_to_string(left_gnode_nodes)

                            # NodeList[St..(k-1)]
                            right_gnode = GNode(parent_name, right_gnode_start, right_gnode_start + gnode_size)
                            right_gnode_nodes = node_list[right_gnode.start:right_gnode.end]
                            right_gnode_str = MDR.gnode_to_string(right_gnode_nodes)
                            
                            # 7) EditDist(NodeList[St..(k-1), NodeList[k..(k+j-1)])
                            edit_distance = normalized_levenshtein.distance(left_gnode_str, right_gnode_str)
                            
                            gnode_pair = GNodePair(left_gnode, right_gnode)
                            self._debug(
                                'gnode pair = dist: {0:!s} = {1:.2f}'.format(gnode_pair, edit_distance), 5
                            )
                            
                            # {gnode_size: {GNode: float}}
                            distances[gnode_size][gnode_pair] = edit_distance
                            
                            # 8) St = k+j
                            left_gnode_start = right_gnode_start
                        else:
                            self._debug("skipped, right node doesn't exist", 5)
                            self._debug(' ')  # todo(prod): remove this
                            
                        self._debug(' ')   # todo(prod): remove this
                        
                else:
                    self._debug("skipped, there are no pairs to look", 3)
                    self._debug(' ')  # todo(prod): remove this
                    
                self._debug(' ')
                
        return dict(distances)
    
    def _find_data_regions(self, node):
        node_name = self.node_namer(node)
        node_depth = MDR.depth(node)
        
        self._debug("in _find_data_regions of `{}`".format(node_name))
        
        # 1) if TreeDepth(Node) => 3 then
        if node_depth >= MDR.MINIMUM_DEPTH:
            
            # 2) Node.DRs = IdenDRs(1, Node, K, T);
            data_regions = self._identify_data_regions(start_index=0, node=node)
            self.data_regions[node_name] = data_regions
            self._debug("`{}`: data regions found:".format(node_name), 1)
            self._debug(self.data_regions[node_name])
            
            # 3) tempDRs = ∅;
            temp_data_regions = set()
            
            # 4) for each Child ∈ Node.Children do
            for child in node.getchildren():
                
                # 5) FindDRs(Child, K, T);
                self._find_data_regions(child)
                
                # 6) tempDRs = tempDRs ∪ UnCoveredDRs(Node, Child);
                uncovered_data_regions = self._uncovered_data_regions(node, child)
                temp_data_regions = temp_data_regions | uncovered_data_regions
            
            self._debug("`{}`: temp data regions: ".format(node_name), 1)
            self._debug(temp_data_regions)
            
            # 7) Node.DRs = Node.DRs ∪ tempDRs
            self.data_regions[node_name] |= temp_data_regions

            self._debug("`{}`: data regions found (FINAL):".format(node_name), 1)
            self._debug(self.data_regions[node_name])
            
        else:
            self._debug(
                "skipped (less than min depth = {}), calling recursion on children...\n".format(self.MINIMUM_DEPTH), 1
            )
            for child in node.getchildren():
                self._find_data_regions(child)        
            
    def _identify_data_regions(self, start_index, node):
        # tag_name = node.attrib[TAG_NAME_ATTRIB]
        # todo(prod): remove this if not needed anymore 
        node_name = self.node_namer(node)
        self._debug("in _identify_data_regions node: {}".format(node_name))
        self._debug("start_index:{}".format(start_index), 1)
        
        node_distances = self.distances.get(node_name)
        
        if not node_distances:        
            self._debug("no distances, returning empty set")
            return set()

        # 1 maxDR = [0, 0, 0];
        max_dr = DataRegion.empty()
        current_dr = DataRegion.empty()

        # 2 for (i = 1; i <= K; i++) /* compute for each i-combination */
        for gnode_size in range(1, self.max_tag_per_gnode + 1):
            self._debug('gnode_size (i): {}'.format(gnode_size), 2)

            # 3 for (f = start; f <= start+i; f++) /* start from each node */
            # for start_gnode_start_index in range(start_index, start_index + gnode_size + 1):
            for first_gn_start_idx in range(start_index, start_index + gnode_size):  
                self._debug('first_gn_start_idx (f): {}'.format(first_gn_start_idx), 3)

                # 4 flag = true;
                dr_has_started = False

                # 5 for (j = f; j < size(Node.Children); j+i)
                # for left_gnode_start in range(start_node, len(node) , gnode_size):
                for last_gn_start_idx in range(
                    # start_gnode_start_index, len(node) - gnode_size + 1, gnode_size
                    first_gn_start_idx + gnode_size, len(node) - gnode_size + 1, gnode_size
                ):
                    self._debug('last_gn_start_idx (j): {}'.format(last_gn_start_idx), 4)

                    # 6 if Distance(Node, i, j) <= T then
                    gn_last = GNode(node_name, last_gn_start_idx, last_gn_start_idx + gnode_size)
                    gn_before_last = GNode(node_name, last_gn_start_idx - gnode_size, last_gn_start_idx)
                    gn_pair = GNodePair(gn_before_last, gn_last)
                    distance = node_distances[gnode_size][gn_pair]
                    self._checked_gnode_pairs[node_name].add(gn_pair)
                 
                    self._debug("gn_pair (bef last, last): {!s} = {:.2f}".format(gn_pair, distance), 5)

                    if distance <= self.edit_distance_threshold:
                        
                        self._debug('dist passes the threshold!'.format(distance), 6)

                        # 7 if flag=true then
                        if not dr_has_started:
                            
                            self._debug('it is the first pair, init the `current_dr`...'.format(distance), 6)
                            
                            # 8 curDR = [i, j, 2*i];
                            # current_dr = DataRegion(gnode_size, first_gn_start_idx - gnode_size, 2 * gnode_size)
                            # current_dr = DataRegion(gnode_size, first_gn_start_idx, 2 * gnode_size)
                            current_dr = DataRegion.binary_from_last_gnode(gn_last)
                            
                            self._debug('current_dr: {}'.format(current_dr), 6)

                            # 9 flag = false;
                            dr_has_started = True

                        # 10 else curDR[3] = curDR[3] + i;
                        else:
                            self._debug('extending the DR...'.format(distance), 6)
                            # current_dr = DataRegion(
                            #     current_dr[0], current_dr[1], current_dr[2] + gnode_size
                            # ) 
                            current_dr = current_dr.extend_one_gnode()
                            self._debug('current_dr: {}'.format(current_dr), 6)

                    # 11 elseif flag = false then Exit-inner-loop;
                    elif dr_has_started:
                        self._debug('above the threshold, breaking the loop...', 6)
                        break
                        
                    self._debug(" ")  # todo(prod): remove this
                    
                if self.keep_all_data_regions and not current_dr.is_empty:
                    self._all_data_regions_found[node_name].add(current_dr)

                # 13 if (maxDR[3] < curDR[3]) and (maxDR[2] = 0 or (curDR[2]<= maxDR[2]) then
                # todo add a criteria that checks the avg distance when n_nodes_covered is the same and it starts at
                # the same node
                current_is_strictly_larger = max_dr.n_nodes_covered < current_dr.n_nodes_covered
                current_starts_at_same_node_or_before = (
                    max_dr.is_empty or current_dr.first_gnode_start_index <= max_dr.first_gnode_start_index
                )
                
                if current_is_strictly_larger and current_starts_at_same_node_or_before: 
                    self._debug('current DR is bigger than max! replacing...', 3)
                    
                    # 14 maxDR = curDR;
                    self._debug('old max_dr: {}, new max_dr: {}'.format(max_dr, current_dr),  3)
                    max_dr = current_dr
                    
                self._debug('max_dr: {}'.format(max_dr),  2)
                self._debug(" ")  # todo(prod): remove this
                
        self._debug("max_dr: {}\n".format(max_dr))
 
        # 16 if ( maxDR[3] != 0 ) then
        if not max_dr.is_empty:
            
            # 17 if (maxDR[2]+maxDR[3]-1 != size(Node.Children)) then
            last_covered_idx = max_dr.last_covered_tag_index
            self._debug("max_dr.last_covered_tag_index: {}".format(last_covered_idx))
            
            if last_covered_idx < len(node) - 1:
                self._debug("calling recursion! \n")
                
                # 18 return {maxDR} ∪ IdentDRs(maxDR[2]+maxDR[3], Node, K, T)
                return {max_dr} | self._identify_data_regions(last_covered_idx + 1, node)

            # 19 else return {maxDR}
            else:
                self._debug("returning {{max_dr}}")
                return {max_dr}

        # 21 return ∅;
        self._debug("max_dr is empty, returning empty set")
        return set()

    def _uncovered_data_regions(self, node, child):
        node_name = self.node_namer(node)
        node_drs = self.data_regions[node_name]
        children_names = [self.node_namer(c) for c in node.getchildren()]
        child_name = self.node_namer(child)
        child_idx = children_names.index(child_name)

        # 1) for each data region DR in Node.DRs do
        for dr in node_drs:
            # 2) if Child in range DR[2] .. (DR[2] + DR[3] - 1) then
            if child_idx in dr:
                # 3) return null
                return set()
        # 4) return Child.DRs
        return self.data_regions[child_name]

    def _find_data_records(self, get_node_by_name: callable):
        self._debug("in _find_data_records")
        
        all_data_regions: Set[DataRegion] = set.union(*self.data_regions.values())
        self._debug("total nb of data regions to check: {}".format(len(all_data_regions)))
        
        for dr in all_data_regions:
            self._debug("data region: {:!S}".format(dr), 1)
            
            method = self._find_records_1 if dr.gnode_size == 1 else self._find_records_n
            self._debug("selected method: `{}`".format(method.__name__), 2)
            self._debug(" ")  # todo(prod) remove me
            
            gnode: GNode
            for gnode in dr:
                method(gnode, get_node_by_name)
                self._debug(" ")  # todo(prod) remove me
    
    def _find_records_1(self, gnode: GNode, get_node_by_name: callable):
        """Finding data records in a one-component generalized node."""
        self._debug('in `_find_records_1` for gnode `{:!S}`'.format(gnode), 2)
        
        parent_node = get_node_by_name(gnode.parent)
        node = parent_node[gnode.start]
        node_name = self.node_namer(node)
        
        node_children_distances = self.distances[node_name].get(1, None)
        
        if node_children_distances is None:
            self._debug("node doesn't have children distances, returning...", 3)
            return 
                
        # 1) If all children nodes of G are similar
        # it is not well defined what "all .. similar" means - I consider that "similar" means "edit_dist < TH"
        #       hyp 1: it means that every combination 2 by 2 is similar
        #       hyp 2: it means that all the computed edit distances (every sequential pair...) is similar
        # for the sake of practicality and speed, I'll choose the hypothesis 2
        all_children_are_similar = all(d <= self.edit_distance_threshold for d in node_children_distances.values())
        
        # 2) AND G is not a data table row then
        node_is_table_row = node.tag == "tr"
        
        if all_children_are_similar and not node_is_table_row:
            self._debug("its children are data records", 3)
            # 3) each child node of R is a data record
            for i in range(len(node)):
                self.data_records.add(DataRecord([GNode(node_name, i, i + 1)]))
                
        # 4) else G itself is a data record.
        else:
            self._debug("it is a data record itself", 3)
            self.data_records.add(DataRecord([gnode]))
            
        # todo: debug this implementation with examples in the technical paper

    def _find_records_n(self, gnode: GNode, get_node_by_name: callable):
        """Finding data records in an n-component generalized node."""
        self._debug('in `_find_records_n` for gnode `{:!S}`'.format(gnode), 2)
        
        parent_node: lxml.html.HtmlElement = get_node_by_name(gnode.parent)
        nodes = parent_node[gnode.start:gnode.end]
        numbers_children = [len(n) for n in nodes]
        childrens_distances = [self.distances[self.node_namer(n)].get(1, None) for n in nodes]
        
        all_have_same_nb_children = len(set(numbers_children)) == 1
        childrens_are_similar = None not in childrens_distances and all(
            all(d <= self.edit_distance_threshold for d in child_distances) for child_distances in childrens_distances
        )
        
        # 1) If the children nodes of each node in G are similar 
        # 1...)   AND each node also has the same number of children then
        if not (all_have_same_nb_children and childrens_are_similar):
            
            # 3) else G itself is a data record.
            self.data_records.add(DataRecord([gnode]))
            
        else:
            # 2) The corresponding children nodes of every node in G form a non-contiguous object description
            n_children = numbers_children[0]
            for i in range(n_children):
                self.data_records.add(DataRecord([
                    GNode(self.node_namer(n), i, i + 1) for n in nodes
                ]))
            # todo check a case like this
            
        # todo: debug this implementation
        
    def get_data_records_as_node_lists(self, root):
        def get_node(node_name):
            tag = node_name.split("-")[0]
            # todo add some security stuff here???
            node = root.xpath("//{tag}[@___tag_name___='{node_name}']".format(tag=tag, node_name=node_name))[0]
            return node
        # return [[get_node(gn.parent)[gn.start:gn.end] for gn in dr] for dr in self.data_records]
        return [[get_node(gn.parent)[gn.start:gn.end] for gn in data_record] for data_record in self.data_records]
    

In [263]:
type(doc)

lxml.html.HtmlElement

In [264]:
%load_ext autoreload
%autoreload 2

folder = '.'
filename = 'tables-1.html'
# filename = 'tables-1-bis.html'
# filename = 'tables-1-bisbis.html'
doc = open_html_document(folder, filename)
dot = html_to_dot(doc, name_option=SEQUENTIAL)
edit_distance_threshold = 0.5
mdr = MDR(
    max_tag_per_gnode=3, 
    edit_distance_threshold=edit_distance_threshold, 
    verbose=MDRVerbosity.only_identify_data_records()
    # verbose=MDRVerbosity.silent()
)
mdr(doc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
>>>>>>>>>>>>>>>>>>>> START PHASE COMPUTE_DISTANCES (0) <<<<<<<<<<<<<<<<<<<<


self.distances

{'body-00000': None,
 'html-00000': None,
 'table-00000': {1: {GN( 0,  1) - GN( 1,  2): 0.33,
                     GN( 1,  2) - GN( 2,  3): 0.15,
                     GN( 2,  3) - GN( 3,  4): 0.15,
                     GN( 3,  4) - GN( 4,  5): 0.11,
                     GN( 4,  5) - GN( 5,  6): 0.17,
                     GN( 5,  6) - GN( 6,  7): 0.16,
                     GN( 6,  7) - GN( 7,  8): 0.07},
                 2: {GN( 0,  2) - GN( 2,  4): 0.23,
                     GN( 1,  3) - GN( 3,  5): 0.14,
                     GN( 2,  4) - GN( 4,  6): 0.16,
                     GN( 3,  5) - GN( 5,  7): 0.13,
                     GN( 4,  6) - GN( 6,  8): 0.14},
                 3: {GN( 0,  3) - GN( 3,  6): 0.21,
                     GN( 1,  4) - GN( 4,  7): 0.13,
                     GN( 2,  5) - GN( 5,  8): 

[DataRecord(GNode(parent='table-00000', start=0, end=1)),
 DataRecord(GNode(parent='table-00000', start=1, end=2)),
 DataRecord(GNode(parent='table-00000', start=2, end=3)),
 DataRecord(GNode(parent='table-00000', start=3, end=4)),
 DataRecord(GNode(parent='table-00000', start=4, end=5)),
 DataRecord(GNode(parent='table-00000', start=5, end=6)),
 DataRecord(GNode(parent='table-00000', start=6, end=7)),
 DataRecord(GNode(parent='table-00000', start=7, end=8))]

In [265]:
mdr.get_data_records_as_node_lists(doc)



[[[<Element tr at 0x7f367de8de08>]],
 [[<Element tr at 0x7f367de8d1d8>]],
 [[<Element tr at 0x7f367de8d228>]],
 [[<Element tr at 0x7f367df074f8>]],
 [[<Element tr at 0x7f367df07548>]],
 [[<Element tr at 0x7f367df077c8>]],
 [[<Element tr at 0x7f367df07688>]],
 [[<Element tr at 0x7f367df075e8>]]]

In [269]:
import copy
import random
 
 
def gen_colors(n):
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    step = 256 / n
    for i in range(n):
        r += step + int(random.random() * 256 * 0.3)
        g += 2 * step + int(random.random() * 256 * 0.3)
        b += 3 * step + int(random.random() * 256 * 0.3)
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append("{:0>2X}{:0>2X}{:0>2X}".format(r,g,b)) 
    return ret


def paint_data_records(mdr, doc):
    doc_copy = copy.deepcopy(doc)
    data_records = mdr.get_data_records_as_node_lists(doc_copy)
    colors = gen_colors(len(data_records))
    for record, color in zip(data_records, colors):
        for gnode in record:
            for e in gnode: 
                e.set("style", e.attrib.get("style", "") + " background-color: #{}!important;".format(color))
    return doc_copy

In [270]:
gen_colors(3)[0]

'1CEA4C'

In [271]:
painted_str = lxml.etree.tostring(paint_data_records(mdr, doc), pretty_print=True).decode("utf-8")
with open("painted.html", "w") as f:
    f.write(painted_str)
    